# S&P 500 Price Analysis and Prediction 📈

In this notebook, I will use **Artificial Intelligence** to analyze historical data from the **S&P 500 (^GSPC)** index and attempt to predict price movements. 

The goal is to move from simple data visualization to a robust **Machine Learning** model, identifying patterns and trends over the last 10 years.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Graphic's style
plt.style.use('fivethirtyeight')

In [ ]:
# Fetching data
df = yf.download('^GSPC', start='2014-01-01', end='2024-12-31', auto_adjust=True)

# FIX: If columns are MultiIndex (common in new yfinance versions), flatten them
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# Displaying the first rows to confirm
df.head()

In [ ]:
# Plotting the historical closing price
plt.figure(figsize=(16,8))
plt.title('S&P 500 Historical Closing Price')
plt.plot(df['Close'])
plt.xlabel('Date')
plt.ylabel('Price USD ($)')
plt.show()

### Technical Indicators: Moving Averages

The crossover between the **50-day** and **200-day Moving Averages (MA)** is a classic indicator in financial markets. 

* **Golden Cross:** When the 50-day MA crosses above the 200-day MA (Bullish signal).
* **Death Cross:** When the 50-day MA crosses below the 200-day MA (Bearish signal).

In [ ]:
# 50-day Moving Average (short/medium term)
df['MA50'] = df['Close'].rolling(window=50).mean()

# 200-day Moving Average (long term - crucial for investors)
df['MA200'] = df['Close'].rolling(window=200).mean()

# Drop rows with NaN values (the first 200 days won't have an average)
df.dropna(inplace=True)

# Plotting the strategy chart
plt.figure(figsize=(16,8))
plt.title('S&P 500: Price vs. Moving Averages (50 & 200 days)')
plt.plot(df['Close'], label='Closing Price', alpha=0.5)
plt.plot(df['MA50'], label='50-Day MA', color='orange')
plt.plot(df['MA200'], label='200-Day MA', color='red')
plt.xlabel('Date')
plt.ylabel('Price USD ($)')
plt.legend()
plt.show()

### Step 3: Target Labeling for Machine Learning

To train a **Binary Classification** model, we need a target variable. 
We will create a 'Target' column where:
* **1 (Up):** If tomorrow's closing price is higher than today's.
* **0 (Down):** If tomorrow's closing price is lower than today's.

We use the `.shift(-1)` method to align tomorrow's price with today's features.

In [ ]:
# Create a column for tomorrow's price
# We use .iloc[:, 0] if 'Close' is still being treated as a single-column DataFrame
df['Tomorrow'] = df['Close'].shift(-1)

# Create the Target: 1 if price goes up, 0 if it goes down
# We use .values to compare only the numbers, avoiding index alignment issues
df['Target'] = (df['Tomorrow'].values > df['Close'].values).astype(int)

# Check the results
df[['Close', 'Tomorrow', 'Target']].head()

In [ ]:
# It's important to check if the data is balanced (roughly 50/50 split)
print("Target Distribution:")
print(df['Target'].value_counts(normalize=True))

# Visualizing the distribution
sns.countplot(x='Target', data=df)
plt.title('Target Distribution (0: Down, 1: Up)')
plt.show()

### Step 4: Feature Engineering - Adding Market Context

To help the model identify patterns, we need to provide more than just the closing price. We will add:
1. **RSI (Relative Strength Index):** Measures if the market is overbought (>70) or oversold (<30).
2. **Daily Returns:** Percentage change in price from the previous day.
3. **Volatility:** The standard deviation of prices over a 10-day window.
4. **Volume Gap:** Percentage change in trading volume.
5. **HL_Range:** The spread between the day's High and Low price.

In [ ]:
# 1. RSI (Relative Strength Index) - 14 periods
delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# 2. Daily Returns (%)
df['Daily_Return'] = df['Close'].pct_change()

# 3. Volatility (10-day rolling standard deviation)
df['Volatility'] = df['Close'].rolling(window=10).std()

# 4. Volume Change (%)
df['Volume_Change'] = df['Volume'].pct_change()

# 5. High-Low Range (Daily spread)
df['HL_Range'] = (df['High'] - df['Low']) / df['Close']

# Drop the new NaN values created by these calculations
df.dropna(inplace=True)

# Display the updated dataframe with the new features
df[['Close', 'RSI', 'Daily_Return', 'Volatility', 'Volume_Change', 'HL_Range']].tail()

### Visualizing Feature Correlation

Before training, it's useful to see how our new features relate to each other. A Heatmap helps us identify if some features are too similar (multicollinearity) or have a visible relationship with the Target.

In [ ]:
plt.figure(figsize=(12, 8))
# We only correlate numeric features and the Target
correlation_matrix = df[['Close', 'MA50', 'MA200', 'RSI', 'Daily_Return', 'Volatility', 'Volume_Change', 'HL_Range', 'Target']].corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Feature Correlation Heatmap')
plt.show()

### Step 5: Training the Random Forest Model

In financial time series, we **cannot** use random shuffling for our training and testing sets, as this would cause "look-ahead bias" (using future data to predict the past). 

Instead, we will:
1. Split the data chronologically (using the first 80% for training and the last 20% for testing).
2. Use a **Random Forest Classifier** with 200 estimators.
3. Evaluate the results using a **Confusion Matrix** and **Classification Report**.

In [ ]:
# 1. Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# 2. Drop all rows that contain any NaN (including the ones we just created)
df.dropna(inplace=True)

# 3. Sanity Check: Ensure there are no more NaNs or Infs
print("Are there any NaNs left?", df.isnull().values.any())
print("Are there any Infs left?", np.isinf(df.to_numpy()).any())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Selecting our features and target
features = ['Close', 'MA50', 'MA200', 'RSI', 'Daily_Return', 'Volatility', 'Volume_Change', 'HL_Range']
X = df[features]
y = df['Target']

# Chronological split (last 20% for testing)
split_index = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

print(f"Training samples: {len(X_train)} | Testing samples: {len(X_test)}")

In [ ]:
# Initializing the model
# n_estimators=200 means we use 200 decision trees
# min_samples_split=100 helps prevent overfitting
model = RandomForestClassifier(n_estimators=200, min_samples_split=100, random_state=1)

# Training the model
model.fit(X_train, y_train)

# Making predictions
preds = model.predict(X_test)

In [ ]:
# Calculating Accuracy
acc = accuracy_score(y_test, preds)
print(f"Model Accuracy: {acc:.2%}")

# Confusion Matrix Visualization
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_test, preds), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix: Predicted vs Actual')
plt.xlabel('Predicted (0=Down, 1=Up)')
plt.ylabel('Actual (0=Down, 1=Up)')
plt.show()

print(classification_report(y_test, preds))

In [ ]:
# Plotting which features influenced the model the most
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)

plt.figure(figsize=(10,6))
importances.plot(kind='bar')
plt.title('Feature Importance (What the AI learned most from)')
plt.show()

### Step 7: Why Accuracy isn't Everything (Precision & Logic)

In trading, we don't need to be right every time. We need to be right when we decide to actually enter a trade. 

* **Precision:** When the model predicted "UP", how often was it actually "UP"?
* **Backtesting:** Let's simulate a simple strategy: if the model predicts a rise, we "buy" the index.

In [ ]:
# 1. Ensure we are using the correct variables from the 48% model
# If your predictions were saved as 'preds', use them here
current_preds = preds 

# 2. Calculate Precision
from sklearn.metrics import precision_score
prec = precision_score(y_test, current_preds)

# 3. Create the Backtest DataFrame
# We use .copy() to avoid warnings and ensure the index matches y_test
backtest_df = pd.DataFrame(index=y_test.index)
backtest_df['Actual_Target'] = y_test.values
backtest_df['Predictions'] = current_preds

# 4. Get the Actual Returns from the original dataframe
# This is where the error often happens, so we use .loc to be precise
backtest_df['Actual_Returns'] = df['Daily_Return'].loc[y_test.index].values

# 5. Calculate Strategy Returns
# If the model predicts 1 (Up), we take the market return. If 0, we stay out (0 return).
backtest_df['Strategy_Returns'] = backtest_df['Predictions'] * backtest_df['Actual_Returns']

# 6. Calculate Cumulative Growth (Starting from $1)
backtest_df['Cumulative_Market'] = (1 + backtest_df['Actual_Returns']).cumprod()
backtest_df['Cumulative_Strategy'] = (1 + backtest_df['Strategy_Returns']).cumprod()

print(f"Model Precision (When it says 'Up', how often it's right): {prec:.2%}")

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Performance: AI Strategy vs. S&P 500 Buy & Hold')
plt.plot(backtest_df['Cumulative_Market'], label='S&P 500 (Market)', color='gray', alpha=0.5)
plt.plot(backtest_df['Cumulative_Strategy'], label='AI Strategy', color='green', linewidth=2)
plt.xlabel('Date')
plt.ylabel('Growth of $1')
plt.legend()
plt.show()

## Final Conclusion 🏁

### 1. The "Quant" Edge
While the cumulative return of this initial model underperformed the **Buy & Hold** strategy, we achieved a **Precision of 55% on 'UP' signals**. In quantitative finance, a consistent 5% edge over random chance is highly significant.

### 2. Why Buy & Hold won?
The S&P 500 is inherently biased to the upside. By staying in "Cash" during the model's 'Down' predictions, we missed some recovery rallies. This is a classic trade-off between **Risk Mitigation** and **Opportunity Cost**.

### 3. Future Improvements
To move this from a 55% precision to a profitable trading system, I would:
* Implement **Sentiment Analysis** (NLP) from financial news.
* Use **Hyperparameter Tuning** (Optuna) to refine the Random Forest.
* Add **Macroeconomic Indicators** (Interest rates, Inflation) as features.

**This notebook demonstrates a full end-to-end Machine Learning pipeline: from data ingestion and cleaning to feature engineering and backtesting.**